In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.losses import categorical_crossentropy
from PIL import Image, ImageOps
import numpy as np
import sklearn.model_selection
import os

In [ ]:
#For the program to work, data needs to be downloaded (https://drive.google.com/file/d/1A3dvLECp6Qva4F3jsYWUL3IXcwqJM4Wj/view?usp=sharing) and loaded after directories are created.

def createDirectories():
  #Directories structure (it is checked if directory already exists in order not to erase uploaded images):
  if not os.path.isdir("baseData"):
    os.mkdir("baseData")
  
  #All images should be uploaded to respective directories (CT_COVID or CT_NonCOVID)
  if not os.path.isdir("baseData/CT_COVID"):
    
    os.mkdir("baseData/CT_COVID")
  if not os.path.isdir("baseData/CT_NonCOVID"):
    os.mkdir("baseData/CT_NonCOVID")
  
  if not os.path.isdir("processedData"):
    os.mkdir("processedData")
    
createDirectories()

In [ ]:
def preprocessImages(img_cols, img_rows):
  #Load data, modify files and save them (COVID)
  for i, fileName in enumerate(os.listdir("baseData/CT_COVID")):
    image = Image.open("baseData/CT_COVID/{}".format(fileName)) #load
    resizedImage = image.resize((img_cols, img_rows)) #resize
    grayImage = ImageOps.grayscale(resizedImage) #grayscale
    grayImage.save("processedData/COVID_{}.jpg".format(i)) #save

  #Load data, modify files and save them (NonCOVID)
  for i, fileName in enumerate(os.listdir("baseData/CT_NonCOVID")):
    image = Image.open("baseData/CT_NonCOVID/{}".format(fileName)) #load
    resizedImage = image.resize((img_cols, img_rows)) #resize
    grayImage = ImageOps.grayscale(resizedImage) #grayscale
    grayImage.save("processedData/NonCOVID_{}.jpg".format(i)) #save

In [ ]:
def createNumpyData(num_classes):
  xData = []
  yData = []
  #Load processed data and append values to arrays
  for fileName in os.listdir("processedData"):
    image = Image.open('processedData/{}'.format(fileName))
    xData.append(np.asarray(image))
    if fileName[0] == "C":
      yData.append(1)
    else:
      yData.append(0)
  
  #Convert python array to numpy array
  xNumpyData = np.array(xData)
  yNumpyData = np.array(yData)
  
  #Convert pixel values to values between 0 and 1
  xNumpyData = xNumpyData.astype('float32')
  xNumpyData /= 255
  
  #Assign classes for yData
  yNumpyData = np_utils.to_categorical(yNumpyData, num_classes)

  return xNumpyData, yNumpyData

In [ ]:
def splitData(xNumpyData, yNumpyData, test_size, img_cols, img_rows):
  #Split the data
  x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(xNumpyData, yNumpyData, test_size=test_size, random_state=42)
  #Reshape the xData (gray scale is used so only 1 number needed to describe each pixel)
  x_train = x_train.reshape(x_train.shape[0], img_cols, img_rows, 1)
  x_test = x_test.reshape(x_test.shape[0], img_cols, img_rows, 1)
  input_shape = (img_cols, img_rows, 1)

  return x_train, x_test, y_train, y_test, input_shape

In [ ]:
#Specify image size and process the images to vectors
img_cols, img_rows = 140, 100
num_classes = 2

preprocessImages(img_cols, img_rows)
xNumpyData, yNumpyData = createNumpyData(num_classes)

In [ ]:
#Specify test size and split the data
test_size = 0.1
x_train, x_test, y_train, y_test, input_shape = splitData(xNumpyData, yNumpyData, test_size, img_cols, img_rows)

In [ ]:
def createModel(input_shape, num_classes): 
  #Create model
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2))) 
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(num_classes, activation='softmax'))
  model.summary()
  model.compile(loss=categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
  return model

In [ ]:
def testModel(model, x_train, y_train, x_test, y_test, batch_size, epochs):
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(x_test, y_test))
  score = model.evaluate(x_test, y_test, verbose=0)
  return score

In [ ]:
#Specify learning parameters and fit the model
batch_size = 32
epochs = 12

model = createModel(input_shape, num_classes)
testModel(model, x_train, y_train, x_test, y_test, batch_size, epochs)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 138, 98, 32)       320       
                                                                 
 conv2d_3 (Conv2D)           (None, 136, 96, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 68, 48, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 68, 48, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 208896)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               26738816  
                                                      

[0.6684826612472534, 0.746666669845581]